
# Mid-Term Assignment-1

#### Team
###### Kishnu Srivastav - 24PGAI0042
###### Praveen Kumar Singh - 24PGAI0068
###### Harshit Mishra - 24PGAI0076
###### Soham Sanyal - 24PGAI0055

In [0]:
# Spark Context Created.. Let's verify it
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
sc = spark.sparkContext


#### Read the customerdata.csv file from local system

In [0]:
%fs ls /FileStore/tables/jio2023/

path,name,size,modificationTime
dbfs:/FileStore/tables/jio2023/bestcaptains/,bestcaptains/,0,0
dbfs:/FileStore/tables/jio2023/bestcaptains2/,bestcaptains2/,0,0
dbfs:/FileStore/tables/jio2023/bestc|aptains/,bestc|aptains/,0,0
dbfs:/FileStore/tables/jio2023/captains_ODI-1.csv,captains_ODI-1.csv,4510,1694579190000
dbfs:/FileStore/tables/jio2023/captains_ODI-2.csv,captains_ODI-2.csv,4510,1694682547000
dbfs:/FileStore/tables/jio2023/captains_ODI.csv,captains_ODI.csv,4510,1694489682000
dbfs:/FileStore/tables/jio2023/captains_Test-1.csv,captains_Test-1.csv,3332,1694579190000
dbfs:/FileStore/tables/jio2023/captains_Test-2.csv,captains_Test-2.csv,3332,1694682547000
dbfs:/FileStore/tables/jio2023/captains_Test.csv,captains_Test.csv,3332,1694489682000
dbfs:/FileStore/tables/jio2023/customerdata.csv,customerdata.csv,4499,1695005686000


In [0]:
customers = sc.textFile( "/FileStore/tables/jio2023/customerdata.csv" )


#### Show first 10 records # Customer_Id Income Age

In [0]:
customers.take(10)

Out[4]: ['1,41100,48.75',
 '2,54100,28.1',
 '3,47800,46.75',
 '4,19100,40.25',
 '5,18200,35.8',
 '6,19800,41.45',
 '7,51500,31.55',
 '8,39900,48.2',
 '9,13000,36.8',
 '10,31900,43.05']



#### Creating list of tuples

In [0]:
def parse_record(record):
    """
    Parses a comma-separated record into a tuple of customer information.

    Args:
        record (str): A comma-separated string containing customer information.

    Returns:
        tuple: A tuple with customer ID as the first element, and a nested tuple with
               income (int) and age (float) as the second element.
               Returns None if parsing fails.
    """
    labels = record.split(',')
    try:
        customer_id = labels[0]
        income = int(labels[1])
        age = float(labels[2])
        return (customer_id, (income, age))
    except (IndexError, ValueError):
        return None

cus_conv = customers.map(lambda line: parse_record(line))


#### Show new formatted records

In [0]:
cus_conv.take(10)

Out[6]: [('1', (41100, 48.75)),
 ('2', (54100, 28.1)),
 ('3', (47800, 46.75)),
 ('4', (19100, 40.25)),
 ('5', (18200, 35.8)),
 ('6', (19800, 41.45)),
 ('7', (51500, 31.55)),
 ('8', (39900, 48.2)),
 ('9', (13000, 36.8)),
 ('10', (31900, 43.05))]


#### Extract income and age values

In [0]:
# Extract income and age values
incomes = cus_conv.map(lambda rec: rec[1][0])
ages = cus_conv.map(lambda rec: rec[1][1])

# Calculate max and min values
max_income = incomes.reduce(max)
min_income = incomes.reduce(min)
max_age = ages.reduce(max)
min_age = ages.reduce(min)

max_age, min_age, max_income, min_income

Out[7]: (50.4, 26.7, 60200, 3100)


####   Normalizes a data point based on income and age using min-max scaling

In [0]:
def normalize_data_points(data_point, min_income, max_income, min_age, max_age):
    """
    Args:
        data_point (tuple): A tuple containing income (int) and age (float).
        min_income (int): Minimum income in the dataset.
        max_income (int): Maximum income in the dataset.
        min_age (float): Minimum age in the dataset.
        max_age (float): Maximum age in the dataset.

    Returns:
        tuple: A tuple with normalized income and age values.
    """
    income, age = data_point
    normalized_income = (income - min_income) / (max_income - min_income)
    normalized_age = (age - min_age) / (max_age - min_age)
    return (normalized_income, normalized_age)

# Assuming 'cus_conv' contains customer data in the format (customer_id, (income, age))
normalized_data_points = cus_conv.map(lambda rec: normalize_data_points(rec[1], min_income, max_income, min_age, max_age))

# Take the first 10 normalized data points
normalized_data_points.take(10)


Out[8]: [(0.6654991243432574, 0.9303797468354431),
 (0.8931698774080561, 0.059071729957806),
 (0.7828371278458844, 0.8459915611814347),
 (0.28021015761821366, 0.5717299578059072),
 (0.26444833625218916, 0.3839662447257383),
 (0.29246935201401053, 0.6223628691983124),
 (0.8476357267950964, 0.20464135021097052),
 (0.6444833625218914, 0.9071729957805909),
 (0.1733800350262697, 0.4261603375527425),
 (0.5043782837127846, 0.6898734177215189)]


#### Broadcast the initial cluster centers to all nodes in the Spark cluster

In [0]:
# Given number of clusters
num_clusters = 5

# Randomly initialize cluster centers
initial_centers = normalized_data_points.takeSample(False, num_clusters)

# Broadcast the initial cluster centers to all nodes in the Spark cluster
initial_centers_broadcast = sc.broadcast(initial_centers)

# Access the value of the broadcasted cluster centers
initial_centers_broadcast.value


Out[9]: [(0.6672504378283712, 0.8691983122362869),
 (0.16637478108581435, 0.5316455696202531),
 (0.6672504378283712, 0.6687763713080168),
 (0.26444833625218916, 0.3839662447257383),
 (0.9176882661996497, 0.10548523206751055)]


####  Calculate the Manhattan distance between two points in a 2D space.

In [0]:
def calculate_manhattan_distance(point_a, point_b):
    """
    Args:
        point_a (tuple): A tuple representing the first point (x, y).
        point_b (tuple): A tuple representing the second point (x, y).

    Returns:
        float: The Manhattan distance between the two points.
    """
    distance = abs(point_a[0] - point_b[0]) + abs(point_a[1] - point_b[1])
    return distance



####    Assign an initial cluster to a data point based on the Manhattan distance to cluster centers.


In [0]:
def assign_initial_cluster_to_datapoint(data_point, initial_centers):
    """
    Args:
        data_point (tuple): A tuple representing a data point (x, y).
        initial_centers (list): A list of initial cluster centers.

    Returns:
        tuple: A tuple with the assigned cluster center and its corresponding Manhattan distance.
    """
    # Calculate Manhattan distances to each initial cluster center
    distances = [(calculate_manhattan_distance(data_point, center), center) for center in initial_centers]

    # Sort distances and return the closest cluster center
    closest_cluster = sorted(distances, key=lambda rec: rec[0])[0]
    return closest_cluster

# Assign initial clusters to normalized data points
initial_centers = initial_centers_broadcast.value
norm_data_clustered = normalized_data_points.map(lambda rec: (assign_initial_cluster_to_datapoint(rec, initial_centers)[1], rec))

# Take the first 10 cluster assignments
norm_data_clustered.take(10)


Out[11]: [((0.6672504378283712, 0.8691983122362869),
  (0.6654991243432574, 0.9303797468354431)),
 ((0.9176882661996497, 0.10548523206751055),
  (0.8931698774080561, 0.059071729957806)),
 ((0.6672504378283712, 0.8691983122362869),
  (0.7828371278458844, 0.8459915611814347)),
 ((0.16637478108581435, 0.5316455696202531),
  (0.28021015761821366, 0.5717299578059072)),
 ((0.26444833625218916, 0.3839662447257383),
  (0.26444833625218916, 0.3839662447257383)),
 ((0.16637478108581435, 0.5316455696202531),
  (0.29246935201401053, 0.6223628691983124)),
 ((0.9176882661996497, 0.10548523206751055),
  (0.8476357267950964, 0.20464135021097052)),
 ((0.6672504378283712, 0.8691983122362869),
  (0.6444833625218914, 0.9071729957805909)),
 ((0.16637478108581435, 0.5316455696202531),
  (0.1733800350262697, 0.4261603375527425)),
 ((0.6672504378283712, 0.6687763713080168),
  (0.5043782837127846, 0.6898734177215189))]


####    Calculate the centroid of a cluster based on its data points.


In [0]:
def calculate_centroid(data_points):
    """
    Args:
        data_points (list): A list of data points represented as tuples.

    Returns:
        tuple: The centroid of the cluster as a tuple.
    """
    num_points = len(data_points)
    if num_points == 0:
        return None

    # Calculate the sum of x and y values for all data points
    sum_x = sum(point[0] for point in data_points)
    sum_y = sum(point[1] for point in data_points)

    # Calculate the centroid by dividing the sums by the number of data points
    centroid = (sum_x / num_points, sum_y / num_points)
    return centroid



#### Calculate new cluster centers based on the clustered data

In [0]:
def calculate_new_centers(clustered_data):
    """
    Calculate new cluster centers based on clustered data.

    Args:
        clustered_data (RDD): An RDD containing clustered data.

    Returns:
        list: A list of tuples representing the new cluster centers.
    """
    # Group data by cluster and calculate the centroid for each cluster
    centers = norm_data_clustered.groupByKey().map(lambda rec: (rec[0], calculate_centroid(list(rec[1]))))

    # Broadcast the new cluster centers to all nodes in the Spark cluster
    new_centers_broadcast = sc.broadcast(centers.collect())
    return new_centers_broadcast.value

new_cluster_centers = calculate_new_centers(norm_data_clustered)
new_cluster_centers

Out[13]: [((0.6672504378283712, 0.8691983122362869),
  (0.7116795930281044, 0.873367490456098)),
 ((0.9176882661996497, 0.10548523206751055),
  (0.9125043782837128, 0.11324894514767936)),
 ((0.16637478108581435, 0.5316455696202531),
  (0.22378224346225767, 0.5690123721662019)),
 ((0.26444833625218916, 0.3839662447257383),
  (0.29516368045264724, 0.3465595585848751)),
 ((0.6672504378283712, 0.6687763713080168),
  (0.6301338907406362, 0.6959303116918472))]


####  Assign a new cluster to a data point based on the Manhattan distance to new cluster centers.

In [0]:
def assign_new_cluster_to_datapoint(data_point, new_centers):
    """

    Args:
        data_point (tuple): A tuple representing a data point (x, y).
        new_centers (list): A list of tuples representing new cluster centers.

    Returns:
        tuple: A tuple with the assigned cluster center and its corresponding Manhattan distance.
    """
    # Calculate Manhattan distances to each new cluster center
    distances = [(calculate_manhattan_distance(data_point[1], center[1]), center[1]) for center in new_centers]

    # Sort distances and return the closest new cluster center
    closest_cluster = sorted(distances, key=lambda rec: rec[0])[0]
    return closest_cluster

# Assign new clusters to data points based on new cluster centers
norm_data_clustered = norm_data_clustered.map(lambda rec: (assign_new_cluster_to_datapoint(rec, new_cluster_centers)[1],rec[1])).sortByKey()

# Take the first 10 data points with new cluster assignments
norm_data_clustered.take(10)


Out[14]: [((0.22378224346225767, 0.5690123721662019),
  (0.28021015761821366, 0.5717299578059072)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.29246935201401053, 0.6223628691983124)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.1733800350262697, 0.4261603375527425)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.26444833625218916, 0.609704641350211)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.3327495621716287, 0.8417721518987342)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.2381786339754816, 0.5696202531645571)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.17688266199649738, 0.5168776371308018)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.1646234676007005, 0.49156118143459926)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.05253940455341506, 0.44725738396624465)),
 ((0.22378224346225767, 0.5690123721662019),
  (0.27495621716287216, 0.7109704641350211))]



#### Perform clustering for a specified number of iterations.


In [0]:
def cluster_results(clustered_data, num_iterations):
    """  
    Args:
        clustered_data (RDD): An RDD containing clustered data.
        num_iterations (int): The number of iterations for clustering.

    Returns:
        RDD: The clustered data after the specified number of iterations.
    """
    for i in range(num_iterations):
        # Calculate new cluster centers
        new_cluster_centers = calculate_new_centers(clustered_data)
        
        # Assign new clusters to data points
        clustered_data = clustered_data.map(lambda rec: (assign_new_cluster_to_datapoint(rec, new_cluster_centers)[1], rec[1]))

    return clustered_data

num_iterations = 30

# Perform clustering for 30 iterations
final_clustered_data = cluster_results(norm_data_clustered, num_iterations)
final_clustered_data.take(10)

Out[15]: [((0.24030143819986197, 0.5786344457230534),
  (0.28021015761821366, 0.5717299578059072)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.29246935201401053, 0.6223628691983124)),
 ((0.28590192644483364, 0.33263009845288327),
  (0.1733800350262697, 0.4261603375527425)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.26444833625218916, 0.609704641350211)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.3327495621716287, 0.8417721518987342)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.2381786339754816, 0.5696202531645571)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.17688266199649738, 0.5168776371308018)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.1646234676007005, 0.49156118143459926)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.05253940455341506, 0.44725738396624465)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.27495621716287216, 0.7109704641350211))]

In [0]:
final_clustered_data.take(300)

Out[16]: [((0.24030143819986197, 0.5786344457230534),
  (0.28021015761821366, 0.5717299578059072)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.29246935201401053, 0.6223628691983124)),
 ((0.28590192644483364, 0.33263009845288327),
  (0.1733800350262697, 0.4261603375527425)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.26444833625218916, 0.609704641350211)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.3327495621716287, 0.8417721518987342)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.2381786339754816, 0.5696202531645571)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.17688266199649738, 0.5168776371308018)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.1646234676007005, 0.49156118143459926)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.05253940455341506, 0.44725738396624465)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.27495621716287216, 0.7109704641350211)),
 ((0.24030143819986197, 0.5786344457230534),
  (0.3117338003502627, 0.5590717299578


#### Denormalize a data point to its original scale based on min-max scaling parameters.


In [0]:
def denormalize_data(data_point, min_inc, max_inc, min_age, max_age):
    """
    Args:
        data_point (tuple): A tuple representing a normalized data point.
        min_inc (int): Minimum income in the original data.
        max_inc (int): Maximum income in the original data.
        min_age (float): Minimum age in the original data.
        max_age (float): Maximum age in the original data.

    Returns:
        tuple: A tuple with denormalized income and age values.
    """
    income, age = data_point
    denormalized_income = round(income * (max_income - min_income) + min_income)
    denormalized_age = round(age * (max_age - min_age) + min_age, 2)
    return (denormalized_income, denormalized_age)

# Denormalize the clustered data to the original scale
final_clustered_data_denorm = final_clustered_data.map(lambda rec: (denormalize_data(rec[0], min_income, max_income, min_age, max_age),
                                                                   denormalize_data(rec[1], min_income, max_income, min_age, max_age)))

# Take the first 5 denormalized clustered data points
final_clustered_data_denorm.take(10)

Out[17]: [((16821, 40.41), (19100, 40.25)),
 ((16821, 40.41), (19800, 41.45)),
 ((19425, 34.58), (13000, 36.8)),
 ((16821, 40.41), (18200, 41.15)),
 ((16821, 40.41), (22100, 46.65)),
 ((16821, 40.41), (16700, 40.2)),
 ((16821, 40.41), (13200, 38.95)),
 ((16821, 40.41), (12500, 38.35)),
 ((16821, 40.41), (6100, 37.3)),
 ((16821, 40.41), (18800, 43.55))]

In [0]:
final_clustered_data_denorm.count(), final_clustered_data_denorm.take(300)

Out[18]: (300,
 [((16821, 40.41), (19100, 40.25)),
  ((16821, 40.41), (19800, 41.45)),
  ((19425, 34.58), (13000, 36.8)),
  ((16821, 40.41), (18200, 41.15)),
  ((16821, 40.41), (22100, 46.65)),
  ((16821, 40.41), (16700, 40.2)),
  ((16821, 40.41), (13200, 38.95)),
  ((16821, 40.41), (12500, 38.35)),
  ((16821, 40.41), (6100, 37.3)),
  ((16821, 40.41), (18800, 43.55)),
  ((16821, 40.41), (20900, 39.95)),
  ((16821, 40.41), (27600, 41.35)),
  ((16821, 40.41), (15700, 40.55)),
  ((16821, 40.41), (15000, 44.15)),
  ((16821, 40.41), (25400, 39.8)),
  ((16821, 40.41), (23200, 43.25)),
  ((16821, 40.41), (3200, 39.65)),
  ((16821, 40.41), (11500, 44.15)),
  ((16821, 40.41), (3700, 40.45)),
  ((16821, 40.41), (3100, 42.6)),
  ((16821, 40.41), (25900, 39.2)),
  ((16821, 40.41), (9900, 37.5)),
  ((16821, 40.41), (9600, 42.4)),
  ((16821, 40.41), (27400, 38.75)),
  ((16821, 40.41), (8900, 41.05)),
  ((16821, 40.41), (12900, 40.7)),
  ((16821, 40.41), (22200, 40.9)),
  ((16821, 40.41), (17900, 38.


#### Customers to retrieve for each cluster. Default is 10.

In [0]:
def get_top_n_customers_per_cluster(clustered_data, n=10):
    """
    Args:
        clustered_data (RDD): An RDD containing clustered data with cluster assignments.
        n (int): The number of top customers to retrieve for each cluster. Default is 10.

    Returns:
        list: A list of tuples where each tuple contains the cluster centroid and the top n customers.
    """
    # Group clustered data by cluster centroid and convert values to lists
    clustered_data_grouped = clustered_data.groupByKey().mapValues(list)

    # Get the top n customers for each cluster
    top_n_customers_per_cluster = clustered_data_grouped.map(lambda x: (x[0], x[1][:n]))

    return top_n_customers_per_cluster.collect()

# Get the top 10 customers for each cluster
top_10_customers_per_cluster = get_top_n_customers_per_cluster(final_clustered_data_denorm, n=10)

# Print the results
for cluster_centroid, top_customers in top_10_customers_per_cluster:
    print(f'Cluster Centroid: {cluster_centroid} -> Top {len(top_customers)} Customers: {top_customers}\n')


Cluster Centroid: (39427, 43.58) -> Top 10 Customers: [(31900, 43.05), (46300, 44.3), (40600, 43.7), (47300, 43.3), (40200, 45.65), (44600, 44.2), (30400, 45.6), (39500, 45.6), (41500, 45.15), (43300, 43.75)]

Cluster Centroid: (44773, 47.43) -> Top 10 Customers: [(41100, 48.75), (47800, 46.75), (39900, 48.2), (46500, 47.2), (46400, 45.15), (46000, 44.8), (41200, 47.3), (40500, 48.65), (41700, 47.6), (41100, 46.6)]

Cluster Centroid: (16821, 40.41) -> Top 10 Customers: [(19100, 40.25), (19800, 41.45), (18200, 41.15), (22100, 46.65), (16700, 40.2), (13200, 38.95), (12500, 38.35), (6100, 37.3), (18800, 43.55), (20900, 39.95)]

Cluster Centroid: (19425, 34.58) -> Top 10 Customers: [(13000, 36.8), (10300, 36.9), (18200, 35.8), (8000, 36.5), (27300, 30.65), (17000, 28.95), (28000, 37.4), (16100, 36.15), (8300, 34.1), (18400, 36.75)]

Cluster Centroid: (55204, 29.38) -> Top 10 Customers: [(54100, 28.1), (51500, 31.55), (51700, 29.7), (56700, 29.25), (58800, 29.65), (55000, 28.35), (53200, 29


#### Collect the clustered data to create a DataFrame

In [0]:
import pandas as pd

# Collect the clustered data to create a DataFrame
clustered_data = final_clustered_data_denorm.collect()

# Create a DataFrame with appropriate column names
clustered_df = pd.DataFrame(clustered_data, columns=['Centroid', 'Income_Age'])

# Display the first few rows of the DataFrame
clustered_df.head()


,Centroid,Income_Age
0,"(16821, 40.41)","(19100, 40.25)"
1,"(16821, 40.41)","(19800, 41.45)"
2,"(19425, 34.58)","(13000, 36.8)"
3,"(16821, 40.41)","(18200, 41.15)"
4,"(16821, 40.41)","(22100, 46.65)"


In [0]:
# Collect the customer data into a DataFrame
customer_data = cus_conv.collect()

# Create a DataFrame with appropriate column names
customer_df = pd.DataFrame(customer_data, columns=['CustomerID', 'Income_Age'])

# Display the first few rows of the DataFrame
customer_df.head()

,CustomerID,Income_Age
0,1,"(41100, 48.75)"
1,2,"(54100, 28.1)"
2,3,"(47800, 46.75)"
3,4,"(19100, 40.25)"
4,5,"(18200, 35.8)"


In [0]:
# Merge customer data (customer_df) and clustered data (clustered_df) on the 'Income_Age' column
merged_data = customer_df.merge(clustered_df, on='Income_Age')

# Remove duplicate rows
merged_data.drop_duplicates(inplace=True)

# Group the merged data by 'Centroid' and aggregate lists of 'CustomerID' and 'Income_Age'
grouped_data = merged_data.groupby('Centroid').agg({'CustomerID': list, 'Income_Age': list})

# Rename the final DataFrame if needed (e.g., df)
final_df = grouped_data

final_df.head()

,CustomerID,Income_Age
Centroid,,
"(16821, 40.41)","[4, 6, 16, 19, 21, 26, 28, 31, 36, 41, 46, 49,...","[(19100, 40.25), (19800, 41.45), (18200, 41.15..."
"(19425, 34.58)","[5, 9, 25, 38, 48, 71, 78, 99, 126, 135, 140, ...","[(18200, 35.8), (13000, 36.8), (8000, 36.5), (..."
"(39427, 43.58)","[10, 18, 27, 33, 221, 43, 67, 72, 84, 90, 100,...","[(31900, 43.05), (46300, 44.3), (40600, 43.7),..."
"(44773, 47.43)","[1, 3, 8, 14, 17, 22, 24, 32, 39, 56, 57, 60, ...","[(41100, 48.75), (47800, 46.75), (39900, 48.2)..."
"(55204, 29.38)","[2, 7, 11, 12, 13, 15, 20, 23, 29, 30, 34, 35,...","[(54100, 28.1), (51500, 31.55), (51700, 29.7),..."



#### Print the final results:
###### a. First 10 customers for each segment
###### b. Centroid values of each segment

In [0]:
# Truncate lists of 'CustomerID' and 'Income_Age' to the first 10 elements
final_df['CustomerID'] = final_df['CustomerID'].apply(lambda x: x[:10])
final_df['Income_Age'] = final_df['Income_Age'].apply(lambda x: x[:10])

# Set the option to display the full content of columns
pd.set_option('display.max_colwidth', None)

final_df.head()

,CustomerID,Income_Age
Centroid,,
"(16821, 40.41)","[4, 6, 16, 19, 21, 26, 28, 31, 36, 41]","[(19100, 40.25), (19800, 41.45), (18200, 41.15), (22100, 46.65), (16700, 40.2), (13200, 38.95), (12500, 38.35), (23900, 38.3), (6100, 37.3), (18800, 43.55)]"
"(19425, 34.58)","[5, 9, 25, 38, 48, 71, 78, 99, 126, 135]","[(18200, 35.8), (13000, 36.8), (8000, 36.5), (27300, 30.65), (17000, 28.95), (28000, 37.4), (16100, 36.15), (8300, 34.1), (18400, 36.75), (17000, 35.55)]"
"(39427, 43.58)","[10, 18, 27, 33, 221, 43, 67, 72, 84, 90]","[(31900, 43.05), (46300, 44.3), (40600, 43.7), (47300, 43.3), (47300, 43.3), (40200, 45.65), (44600, 44.2), (30400, 45.6), (39500, 45.6), (41500, 45.15)]"
"(44773, 47.43)","[1, 3, 8, 14, 17, 22, 24, 32, 39, 56]","[(41100, 48.75), (47800, 46.75), (39900, 48.2), (46500, 47.2), (46400, 45.15), (46000, 44.8), (41200, 47.3), (40500, 48.65), (41700, 47.6), (41100, 46.6)]"
"(55204, 29.38)","[2, 7, 11, 12, 13, 15, 20, 23, 29, 30]","[(54100, 28.1), (51500, 31.55), (51700, 29.7), (56700, 29.25), (58800, 29.65), (55000, 28.35), (53200, 29.0), (53200, 29.2), (56700, 29.85), (56900, 30.8)]"
